In [ ]:
# Basic code

import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity, touches
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 25
scale_factor = Decimal("1")

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))
        

    def clone(self) -> "ChristmasTree":
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )    

def get_tree_list_side_lenght(tree_list: list[ChristmasTree]) -> Decimal:
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_total_score(dict_of_side_length: dict[str, Decimal]):
    score = 0
    for k, v in dict_of_side_length.items():
        score += v ** 2 / Decimal(k)
    return score

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def plot_layout(ax, placed_trees, title):
    num_trees = len(placed_trees)
    colors = plt.cm.viridis([i / num_trees for i in range(num_trees)])

    all_polygons = [t.polygon for t in placed_trees]
    bounds = unary_union(all_polygons).bounds

    for i, tree in enumerate(placed_trees):
        # Rescale for plotting
        x_scaled, y_scaled = tree.polygon.exterior.xy
        x = [Decimal(val) / scale_factor for val in x_scaled]
        y = [Decimal(val) / scale_factor for val in y_scaled]
        ax.plot(x, y, color=colors[i])
        ax.fill(x, y, alpha=0.5, color=colors[i])

    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor

    width = maxx - minx
    height = maxy - miny
    # side_length = max(width, height)

    side_length = width if width > height else height
    

    square_x = minx if width >= height else minx - (side_length - width) / 2
    square_y = miny if height >= width else miny - (side_length - height) / 2
    bounding_square = Rectangle(
        (float(square_x), float(square_y)),
        float(side_length),
        float(side_length),
        fill=False,
        edgecolor='red',
        linewidth=2,
        linestyle='--',
    )
    ax.add_patch(bounding_square)

    padding = 0.5
    ax.set_xlim(
        float(square_x - Decimal(str(padding))),
        float(square_x + side_length + Decimal(str(padding))))
    ax.set_ylim(float(square_y - Decimal(str(padding))),
                float(square_y + side_length + Decimal(str(padding))))
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')
    ax.set_title(f'{title}. Side: {side_length:0.8f}')

def plot_difference(layout1, layout2):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    plot_layout(ax1, layout1, "Orininal")
    plot_layout(ax2, layout2, "After fix")
    plt.tight_layout()
    plt.show()

def parse_csv(csv_path) -> dict[str, list[ChristmasTree]]:
    print(f'parse_csv: {csv_path=}')

    result = pd.read_csv(csv_path)
    result['x'] = result['x'].str.strip('s')
    result['y'] = result['y'].str.strip('s')
    result['deg'] = result['deg'].str.strip('s')
    result[['group_id', 'item_id']] = result['id'].str.split('_', n=2, expand=True)

    dict_of_tree_list = {}
    dict_of_side_length = {}
    for group_id, group_data in result.groupby('group_id'):
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg']) for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
        dict_of_side_length[group_id] = get_tree_list_side_lenght(tree_list)

    return dict_of_tree_list, dict_of_side_length

def copy_layout(layout):
    return [tree.clone() for tree in layout]

In [ ]:
# Load current best solution
squeezed_solution_path = '/kaggle/input/manual-tree-shifter-and-checker-sa/submission.csv'
dict_of_tree_list, dict_of_side_length = parse_csv(squeezed_solution_path)

# Calculate current total score
squeezed_score = get_total_score(dict_of_side_length)
print(f'\n{squeezed_score=:0.8f}')

Let's find if we have a good solution for smaller layout within bigger layouts.
Full search is quite complicated and long to run, but most problems are always with the corner, so if we check smaller squares for each corner, we check 99% of posible positive cases.

In [ ]:
layout_id = '111'
layout = dict_of_tree_list[layout_id]

# Select corner
all_polygons = [t.polygon for t in layout]
bounds = unary_union(all_polygons).bounds

for corner_x, corner_y in ((bounds[0],bounds[1]), (bounds[0],bounds[3]), (bounds[2],bounds[1]), (bounds[2],bounds[3])):
    print(f'Corner: {corner_x:0.4f}, {corner_y:0.4f}')

    # Create disctionary - biggest distance -> tree
    candidates = {
        max(
            abs(tree.polygon.bounds[0] - corner_x), # distance by x
            abs(tree.polygon.bounds[2] - corner_x), # distance by x
            abs(tree.polygon.bounds[1] - corner_y), # distance by y
            abs(tree.polygon.bounds[3] - corner_y), # distance by y
        ):tree for tree in layout} 

    # Sort by distance (key)
    candidates = dict(sorted(candidates.items()))

    # Reconstruct by order and compare with existing one
    layout_new = []
    for tree in candidates.values():
        layout_new.append(tree)
        layout_new_size = len(layout_new)
        layout_new_size_key = f'{layout_new_size:03n}'

        best_side = dict_of_side_length[layout_new_size_key]
        candidate_side = get_tree_list_side_lenght(layout_new)
        if candidate_side < best_side:
            print(f'   BINGO!!!! {layout_new_size} {best_side:0.8f} -> {candidate_side:0.8f} ({best_side - candidate_side:0.8f})')
            dict_of_tree_list[layout_new_size_key] = copy_layout(layout_new)
            dict_of_side_length[layout_new_size_key] = candidate_side

In [ ]:
# Calculate current total score
squeezed_score = get_total_score(dict_of_side_length)
print(f'\n{squeezed_score=:0.8f}')

In [ ]:
# Save results
tree_data = []
for group_name, tree_list in dict_of_tree_list.items():
    for item_id, tree in enumerate(tree_list):
        tree_data.append({
            'id': f'{group_name}_{item_id}',
            'x': f's{tree.center_x}',
            'y': f's{tree.center_y}',
            'deg': f's{tree.angle}'
        })
tree_data = pd.DataFrame(tree_data)
tree_data.to_csv('submission.csv', index=False)   

# Visual demo for the same (good case is last picture):


In [ ]:
# Load current best solution
dict_of_tree_list, dict_of_side_length = parse_csv(squeezed_solution_path)

# Calculate current total score
squeezed_score = get_total_score(dict_of_side_length)
print(f'\n{squeezed_score=:0.8f}')

In [ ]:

layout_id = '111'
layout = dict_of_tree_list[layout_id]

# Select corner
all_polygons = [t.polygon for t in layout]
bounds = unary_union(all_polygons).bounds

for corner_x, corner_y in ((bounds[2],bounds[1]),):

    # Create disctionary - biggest distance -> tree
    candidates = {
        max(
            abs(tree.polygon.bounds[0] - corner_x), # distance by x
            abs(tree.polygon.bounds[2] - corner_x), # distance by x
            abs(tree.polygon.bounds[1] - corner_y), # distance by y
            abs(tree.polygon.bounds[3] - corner_y), # distance by y
        ):tree for tree in layout} 

    # Sort by distance (key)
    candidates = dict(sorted(candidates.items()))

    # Reconstruct by order and compare with existing one
    layout_new = []
    for dist, tree in candidates.items():
        layout_new.append(tree)
        layout_new_size = len(layout_new)
        layout_new_size_key = f'{layout_new_size:03n}'

        best_side = dict_of_side_length[layout_new_size_key]
        candidate_side = get_tree_list_side_lenght(layout_new)

        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 6))
        
        plot_layout(ax1, layout, f"Big layout, Patch={dist:0.4f}")
        sample = Rectangle(
            (corner_x - dist, corner_y),
            dist, dist,
            edgecolor='blue',
            facecolor='none',
            linewidth=3,     
            linestyle='--'   
        )
        ax1.add_patch(sample)

        plot_layout(ax2, layout_new, "Patch")
        plot_layout(ax3, dict_of_tree_list[layout_new_size_key], "Small layout")

        
        plt.tight_layout()
        plt.show()

        if candidate_side < best_side:
            print(f'   BINGO!!!! {layout_new_size} {best_side:0.8f} -> {candidate_side:0.8f} ({best_side - candidate_side:0.8f})')
            dict_of_tree_list[layout_new_size_key] = copy_layout(layout_new)
            break